# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
import numpy as np

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y =loans_df["credit_ranking"]

# Display a sample of y
np.array(y)

array([0, 0, 0, ..., 1, 0, 1])

In [6]:
# Define features set X by selecting all columns but credit_ranking
X=loans_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
display (X.head())


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [9]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [10]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Transform the training and test datasets
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Display the first few rows of the scaled training data
print(pd.DataFrame(X_train_scaled, columns=X_train.columns).head())

   payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0        -0.733079            0.664893          -1.257044    -0.320459   
1         1.067741           -0.623462           1.523148     0.608863   
2        -1.746040           -1.071585          -1.358142    -0.534917   
3        -0.620528            0.496847          -1.054849    -0.034513   
4        -0.507977            0.608877          -1.004300    -0.534917   

   alumni_success  study_major_code  time_to_completion  \
0       -0.453622         -0.742407           -0.645507   
1       -0.369546         -1.125190           -1.112003   
2       -0.789922          1.075808            0.536281   
3       -0.201396          0.118853            1.189375   
4        0.261018         -0.742407           -0.707707   

   finance_workshop_score  cohort_ranking  total_loan_score  \
0                0.240001        0.988460          0.063095   
1                0.187899       -1.753513         -0.173904   
2          

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [11]:
# Define the the number of inputs (features) to the model

num_features = X_train.shape[1]

# Review the number of features
X_train.shape[1]


11

In [13]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 16

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 8


# Define the number of neurons in the output layer
output_neurons = 1


In [14]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer

model.add(Dense(units=hidden_nodes_layer1, activation='relu', input_dim=num_features))

# Add the second hidden layer
model.add(Dense(units=hidden_nodes_layer2, activation='relu'))


# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=output_neurons, activation='sigmoid'))

In [15]:
# Display the Sequential model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                192       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 337 (1.32 KB)
Trainable params: 337 (1.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [16]:
# Compile the Sequential model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [17]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50
32/32 [==============================] - 3s 25ms/step - loss: 0.7800 - accuracy: 0.4545 - val_loss: 0.7124 - val_accuracy: 0.5273
Epoch 2/50
32/32 [==============================] - 0s 5ms/step - loss: 0.6838 - accuracy: 0.5738 - val_loss: 0.6553 - val_accuracy: 0.6562
Epoch 3/50
32/32 [==============================] - 0s 7ms/step - loss: 0.6427 - accuracy: 0.6725 - val_loss: 0.6297 - val_accuracy: 0.6758
Epoch 4/50
32/32 [==============================] - 0s 8ms/step - loss: 0.6169 - accuracy: 0.6882 - val_loss: 0.6072 - val_accuracy: 0.6797
Epoch 5/50
32/32 [==============================] - 0s 9ms/step - loss: 0.5939 - accuracy: 0.7087 - val_loss: 0.5857 - val_accuracy: 0.6953
Epoch 6/50
32/32 [==============================] - 1s 15ms/step - loss: 0.5724 - accuracy: 0.7253 - val_loss: 0.5677 - val_accuracy: 0.6992
Epoch 7/50
32/32 [==============================] - 0s 14ms/step - loss: 0.5559 - accuracy: 0.7204 - val_loss: 0.5516 - val_accuracy: 0.7070
Epoch 8/50
32/32 

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [19]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)


# Display the model loss and accuracy results
print(f"Loss: {loss}, Accuracy: {accuracy}")


10/10 - 0s - loss: 0.5211 - accuracy: 0.7375 - 70ms/epoch - 7ms/step
Loss: 0.521062433719635, Accuracy: 0.737500011920929


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [20]:
# Set the model's file path
file_path = Path("student_loans.keras")


# Export your model to a keras file
model.save(file_path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [21]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [23]:
# Make predictions with the test data
predictions = model.predict(X_test_scaled)


# Display a sample of the predictions
sample_predictions = predictions[:5]
print(sample_predictions)

10/10 [==============================] - 0s 3ms/step
[[0.3063279 ]
 [0.3252616 ]
 [0.89652914]
 [0.8212523 ]
 [0.97474885]]


In [24]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=['Prediction'], data=predictions)
predictions_df["Prediction"] = round(predictions_df["Prediction"],0)
predictions_df

,Prediction
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
315,1.0
316,0.0
317,0.0
318,1.0


from matplotlib import pyplot as plt
predictions_df['Prediction'].plot(kind='hist', bins=20, title='Prediction')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
predictions_df['Prediction'].plot(kind='line', figsize=(8, 4), title='Prediction')
plt.gca().spines[['top', 'right']].set_visible(False)

### Step 4: Display a classification report with the y test data and predictions

In [27]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["Prediction"].values))

              precision    recall  f1-score   support

           0       0.71      0.76      0.74       154
           1       0.76      0.72      0.74       166

    accuracy                           0.74       320
   macro avg       0.74      0.74      0.74       320
weighted avg       0.74      0.74      0.74       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

Collect student demographics, academic info, loan history, financial aid details, and employment status. This data helps assess socio-economic background, academic performance, creditworthiness, current financial support, and future income potential. Also consider loan preferences. This comprehensive approach allows us to recommend suitable loan options based on each student's unique financial situation, academic prospects, and borrowing behavior.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

The dataset contains comprehensive student profiles, including academic records, financial information, and personal preferences. This allows for tailored loan recommendations based on individual circumstances. Content-based filtering is the preferred method, as it can make personalized suggestions without relying on data from similar users. While collaborative filtering could be used as a supplementary approach if sufficient loan interaction data exists, content-based filtering is more appropriate given the diverse and detailed nature of the available student attributes.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

Data privacy and security are critical when handling sensitive student information. Robust protection measures, including encryption, access controls, and compliance with regulations like GDPR and FERPA, are essential. Proper data handling builds trust and encourages system adoption, while breaches risk legal issues and reputational damage. The recommendation system must also be highly adaptable to changing student circumstances. Financial needs can fluctuate rapidly due to various factors, requiring the system to update quickly and provide accurate, timely recommendations. A static system risks offering outdated advice, potentially causing financial strain or missed opportunities for students. Real-time updates and continuous learning are crucial for maintaining relevance and user satisfaction. Addressing these challenges ensures data integrity and more effective loan recommendations, ultimately better serving students' financial needs.